In [ ]:
import panel as pn
import jsonschema
import json
from panel.widgets import TextInput, Button, Select, IntInput
from IPython.display import display, clear_output

margin = (10, 10, 5, 10)

class Metadata:

    def __init__(self, schema):

        self.schema = schema

        pn.extension()
        self._form_box = pn.WidgetBox()

        self.create_widgets(schema)

        display(self._form_box)

    def create_widgets(self, schema):

        for key, value in schema["properties"].items():
            self._form_box.append(self.create_input_widget(value))


    def create_input_widget(self, schema):
        if "enum" in schema:
            return Select(name=schema["title"], options=schema["enum"], margin=margin)
        elif schema["type"] == "array":
            return self.genetate_array_widget(schema)
        elif schema["type"] == "string":
            return TextInput(name=schema["title"], margin=margin)
        elif schema["type"] == "number":
            return IntInput(name=schema["title"], margin=margin)
        else:
            return None

    def genetate_array_widget(self, schema):
        box =  pn.WidgetBox(margin=margin)
        box.append(schema["title"])

        column = pn.Column(margin=margin)

        def create_items():
            column_list = column.objects
            title = schema["title"] + str(len(column_list) + 1)
            items = schema["items"]
            if items["type"] == "object":
                obj_box = pn.WidgetBox()
                obj_box.append(title)
                for key, value in items["properties"].items():
                    obj_box.append(self.create_input_widget(value))
                return obj_box
            else:
                return self.create_input_widget({
                    "type": items["type"],
                    "title": title
                })

        def add_item(event=None):
            new_widget = create_items()
            row = pn.Row(new_widget)
            column.append(row)

        add_item()
        box.append(column)

        add_button = Button(name='Add', button_type='primary' ,button_style='outline')
        add_button.on_click(add_item)
        box.append(add_button)

        return box